In [1]:
import pandas as pd
import numpy as np

In [2]:
# load data
data = pd.read_csv('cars.csv')
print(data.shape)

(38531, 30)


In [3]:
# check missing and duplicated data
print(f"Missing values: {data.isna().sum().sum()}")
print(f"Duplicate values: {data.duplicated().sum()}")

Missing values: 10
Duplicate values: 40


**The missing data and duplicate data only make up a small portion of the whole dataset, so let's just delete them.**

In [4]:
# delete missing and duplicated data
data.dropna(inplace=True)
data.drop_duplicates(inplace=True)
print(data.shape)

(38481, 30)


In [5]:
data.head()

,manufacturer_name,model_name,transmission,color,odometer_value,year_produced,engine_fuel,engine_has_gas,engine_type,engine_capacity,...,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,duration_listed
0,Subaru,Outback,automatic,silver,190000,2010,gasoline,False,gasoline,2.5,...,True,True,True,False,True,False,True,True,True,16
1,Subaru,Outback,automatic,blue,290000,2002,gasoline,False,gasoline,3.0,...,True,False,False,True,True,False,False,False,True,83
2,Subaru,Forester,automatic,red,402000,2001,gasoline,False,gasoline,2.5,...,True,False,False,False,False,False,False,True,True,151
3,Subaru,Impreza,mechanical,blue,10000,1999,gasoline,False,gasoline,3.0,...,False,False,False,False,False,False,False,False,False,86
4,Subaru,Legacy,automatic,black,280000,2001,gasoline,False,gasoline,2.5,...,True,False,True,True,False,False,False,False,True,7


**The first thing I noticed is that `engine_fuel` and `engine_type` are pretty similar. I choose to check their data.**

In [6]:
print(data.engine_fuel.unique())
print(data.engine_type.unique())

['gasoline' 'gas' 'diesel' 'hybrid-petrol' 'hybrid-diesel']
['gasoline' 'diesel']


In [7]:
mapping = {'gasoline': 'gasoline',
          'gas': 'gasoline',
          'diesel': 'diesel',
          'hybrid-petrol': 'gasoline',
          'hybrid-diesel' : 'diesel',
          'electric': 'electric'}

count = 0
for i in range(len(data)):
    if data.iloc[i].engine_type != mapping[data.iloc[i].engine_fuel]:
        count += 1
print(count)

0


**The count value 0 means they just have this mapping relationship. But `engine_fuel` is more specific, so I will keep it and delete the other.**

In [8]:
data.drop(columns=['engine_type'], inplace=True)

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38481 entries, 0 to 38530
Data columns (total 29 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   manufacturer_name  38481 non-null  object 
 1   model_name         38481 non-null  object 
 2   transmission       38481 non-null  object 
 3   color              38481 non-null  object 
 4   odometer_value     38481 non-null  int64  
 5   year_produced      38481 non-null  int64  
 6   engine_fuel        38481 non-null  object 
 7   engine_has_gas     38481 non-null  bool   
 8   engine_capacity    38481 non-null  float64
 9   body_type          38481 non-null  object 
 10  has_warranty       38481 non-null  bool   
 11  state              38481 non-null  object 
 12  drivetrain         38481 non-null  object 
 13  price_usd          38481 non-null  float64
 14  is_exchangeable    38481 non-null  bool   
 15  location_region    38481 non-null  object 
 16  number_of_photos   384

**In my opinion, `location_region` and `number_of_photos` have no relevance with the `price_usd`, so I prefer to delete them.**

In [10]:
data.drop(columns=['location_region', 'number_of_photos'], inplace=True)

**It's easy to see that many features are boolean type, so we can convert them into 1 and 0.**

In [11]:
for column in data.columns:
    if data.dtypes[column] == 'bool':
        data[column] = data[column].astype(np.int)

**We can further check the object type features.**

In [12]:
{column: len(data[column].unique()) for column in data.columns if data.dtypes[column] == 'object'}

{'manufacturer_name': 55,
 'model_name': 1116,
 'transmission': 2,
 'color': 12,
 'engine_fuel': 5,
 'body_type': 12,
 'state': 3,
 'drivetrain': 3}

In [17]:
print(f"Drivetrain has {data.drivetrain.unique()}")
print(f"Transmission has {data.transmission.unique()}")
print(f"State has {data.state.unique()}")
print(f"Color has {data.color.unique()}")
print(f"Body_type has {data.body_type.unique()}")

Drivetrain has ['all' 'front' 'rear']
Transmission has ['automatic' 'mechanical']
State has ['owned' 'emergency' 'new']
Color has ['silver' 'blue' 'red' 'black' 'grey' 'other' 'brown' 'white' 'green'
 'violet' 'orange' 'yellow']
Body_type has ['universal' 'suv' 'sedan' 'hatchback' 'liftback' 'minivan' 'minibus'
 'van' 'pickup' 'coupe' 'cabriolet' 'limousine']


## My thought:
### Identify groups of features in your data that should be combined into cross-product features. Provide justification for why these features should be crossed (or why some features should not be crossed). 

**The manufacturer_name should be combined with model. These two together make sense. (I can later write explanation)**\
**Drive train and transmission together**\
**Model and body_type. or manufacturer_name, model, body_type**\
**color is not a big deal to me, we can just do like one_hot or later delete it**

**Manufacture: manufacturer_name, model_name, body_type**\
**Power: transmission, engine_fuel, drive train**\
**Appearance: State, color ???**

### metric(s)
If not accuracy. We are doing regression? Maybe others.

## dividing your data into training and testing (i.e., are you using Stratified 10-fold cross validation? Shuffle splits? Why?). Explain why your chosen method is appropriate or use more than one method as appropriate. Argue why your cross validation method is a realistic mirroring of how an algorithm would be used in practice. 

Need to cover all the categories.